# YoloV5 Test
## Generate Image Dataset
### Imports

In [1]:
import cv2
import torch
import os
import numpy as np
from modules import image_processing as ip

### Variable Settings

In [2]:
img_size= 240

img_folder_path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\YoloObjektDetection\\Images\\Dataset\\Srews_Nuts_Washers_Overlaps'
image_name = '1.jpg'
image_path = img_folder_path + '\\' + image_name

model_weights_path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\yolov5_temp\\runs\\train\\Big\\weights\\best.pt'

labels = ['OverlapingObjects', 'Nut', 'Screw', 'Washer']

### Load YoloV5-Model pretrained with own data

In [3]:
def load_model(path):
    # Lade das trainierte Modell
    print(path)
    #model = torch.load(model_weights_path, map_location='cpu')
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=path)
    return model


model = load_model(path=model_weights_path)

C:\Users\doett\Documents\Workspace\GitHub\proj-sorting-robot\Testing\yolov5_temp\runs\train\Big\weights\best.pt


Using cache found in C:\Users\doett/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-12-17 Python-3.11.5 torch-2.1.2+cpu CPU

Fusing layers... 
Model summary: 322 layers, 86193601 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


### Load Images

In [4]:
image_path =  'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\YoloObjektDetection\\Images\\Dataset\\Srews_Nuts_Washers_Overlaps\\1.jpg'

image = cv2.imread(image_path)

# Überprüfen, ob das Bild erfolgreich geladen wurde
if image is not None:
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("[WARNING] No image loaded")
    

### Load more files of one Folder

In [123]:
img_folder_path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\YoloObjektDetection\\Images\\Dataset\\Srews_Nuts_Washers_Overlaps'
img_array = []
# Überprüfen, ob der Ordner existiert
if os.path.isdir(img_folder_path):
    # Durchlaufe alle Dateien im Ordner
    for filename in os.listdir(img_folder_path):
        # Pfade für jedes Object erstellen
        image_path = os.path.join(img_folder_path, filename)

        # Überprüfen, ob es sich um eine Datei handelt
        if os.path.isfile(image_path):
            # Load Image
            image = cv2.imread(image_path)

            # Überprüfen, ob das Bild erfolgreich geladen wurde
            if image is not None:
                img_array.append((image, filename))
                cv2.imshow('Image', image)
            else:
                print(f"Unable to load {filename}")
        else:
            print(f"No File: {filename} ")
else:
    print("Path don't exists")

cv2.waitKey(0)
cv2.destroyAllWindows()

Unable to load 1.json
Unable to load 10.json
Unable to load 11.json
Unable to load 12.json
Unable to load 13.json
Unable to load 14.json
Unable to load 15.json
Unable to load 16.json
Unable to load 17.json
Unable to load 18.json
Unable to load 19.json
Unable to load 2.json
Unable to load 20.json
Unable to load 21.json
Unable to load 22.json
Unable to load 23.json
Unable to load 24.json
Unable to load 25.json
Unable to load 26.json
Unable to load 27.json
Unable to load 28.json
Unable to load 29.json
Unable to load 3.json
Unable to load 30.json
Unable to load 31.json
Unable to load 32.json
Unable to load 33.json
Unable to load 34.json
Unable to load 35.json
Unable to load 36.json
Unable to load 37.json
Unable to load 38.json
Unable to load 39.json
Unable to load 4.json
Unable to load 40.json
Unable to load 41.json
Unable to load 42.json
Unable to load 43.json
Unable to load 44.json
Unable to load 45.json
Unable to load 46.json
Unable to load 47.json
Unable to load 48.json
Unable to load 

In [10]:
imageNr = 0

def get_image(imagenr):
    return img_array[imagenr][0]

def get_image_name(imagenr):
    return img_array[imagenr][1]

print(get_image_name(imageNr))
print(get_image(imageNr))

cv2.imshow('{}'.format(get_image_name(imageNr)), get_image(imageNr))
cv2.waitKey(0)
cv2.destroyAllWindows()


1.jpg
[[[18 20  8]
  [19 21  9]
  [22 22 10]
  ...
  [58 37 10]
  [52 26  9]
  [45 19  3]]

 [[18 20  8]
  [19 21  9]
  [22 22 10]
  ...
  [58 37 10]
  [52 26  9]
  [45 19  3]]

 [[18 20  8]
  [19 21  9]
  [22 22 10]
  ...
  [58 37 10]
  [52 26  9]
  [45 19  3]]

 ...

 [[15  8 11]
  [16  9 12]
  [17 10 13]
  ...
  [ 5 13 12]
  [ 5 13 12]
  [ 3 14 12]]

 [[13  6 11]
  [14  7 12]
  [16  9 14]
  ...
  [ 6 14 13]
  [ 3 14 12]
  [ 1 14 12]]

 [[14  7 12]
  [15  8 13]
  [17 10 15]
  ...
  [ 5 16 14]
  [ 4 15 13]
  [ 1 14 12]]]


### Predict Results

In [39]:
image = cv2.imread(image_path)
image = image[175:950, 0:1700]
output = image.copy()
results = model(image)

cropped_boxes = []  # Liste für ausgeschnittene Boxen
con_array = []
lengh_array = []

for box in results.xyxy[0].cpu().numpy():
    box = [float(i) for i in box]
    xmin, ymin, xmax, ymax, conf, cls = box
    
    region = image[int(ymin):int(ymax), int(xmin):int(xmax)]
    cvt = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(cvt, 13)
    
    _, otsu_thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(otsu_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sortiere Konturen nach Fläche
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Nutze nur die größte Kontur
    if contours:
        contour = contours[0]
        #print(f'contour {contour}')

        adjusted_contour = contour.copy()
        for i, point in enumerate(adjusted_contour):
            adjusted_contour[i][0][0] += xmin  # Verschiebe x-Wert um xmin
            adjusted_contour[i][0][1] += ymin  # Verschiebe y-Wert um ymin
        
        con_array.append(adjusted_contour)
        
        # Berechne das rotierte Rechteck
        rect = cv2.minAreaRect(adjusted_contour)
        box_points = cv2.boxPoints(rect)
        box_points = np.int0(box_points)
        
        # Draw Red Box arround eatch item
        cv2.drawContours(output, [box_points], 0, (0, 0, 255), 2)
        
    output = cv2.rectangle(output, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
    output = cv2.putText(output, labels[int(cls)], (int(xmin), int(ymin) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                         (0, 255, 0), 1, cv2.LINE_AA)    
    
rectangles = ip.get_rects_from_contours(con_array)

bounding_boxes = ip.get_bounding_boxes_from_rectangles(rectangles)

object_images = ip.warp_objects_horizontal(image, rectangles, bounding_boxes)
    
for i,_ in enumerate(object_images):
    lengh_array.append(max(object_images[i].shape))

'''
for i,image in enumerate(object_images):
    cv2.imshow('Single Object',object_images[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()
'''

standardized = ip.standardize_images(object_images)

'''
for i,image in enumerate(standardized):
    cv2.imshow('Single Object',img_array[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()
'''

for i, image in enumerate(standardized):
    path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\Scanned Data\\Item{}.png'.format(i)
    cv2.imwrite(path, image)
    
cv2.imshow("Predicted Objects", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\doett\AppData\Local\Temp\ipykernel_8396\3208511053.py:40: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box_points = np.int0(box_points)


Default Shape(31, 232, 3)
Lengh232
Default Shape(33, 161, 3)
Lengh161
Default Shape(57, 222, 3)
Lengh222
Default Shape(17, 214, 3)
Lengh214
Default Shape(28, 137, 3)
Lengh137
Default Shape(12, 186, 3)
Lengh186
Default Shape(22, 391, 3)
Lengh391
Default Shape(40, 163, 3)
Lengh163
Default Shape(10, 190, 3)
Lengh190
Default Shape(18, 228, 3)
Lengh228
Default Shape(30, 172, 3)
Lengh172
Default Shape(50, 93, 3)
Lengh93
Default Shape(31, 180, 3)
Lengh180
Default Shape(30, 100, 3)
Lengh100
Default Shape(39, 188, 3)
Lengh188
Default Shape(31, 157, 3)
Lengh157
Default Shape(30, 159, 3)
Lengh159
Default Shape(30, 96, 3)
Lengh96
Default Shape(23, 371, 3)
Lengh371
Default Shape(22, 236, 3)
Lengh236
Default Shape(39, 258, 3)
Lengh258
Default Shape(37, 37, 3)
Lengh37
Default Shape(42, 43, 3)
Lengh43
Default Shape(24, 389, 3)
Lengh389
Default Shape(29, 30, 3)
Lengh30
Default Shape(30, 30, 3)
Lengh30


In [18]:
image = cv2.imread(image_path)
image = image[175:950, 0:1700]
output = image.copy()
results = model(image)
cnt = 0

cropped_boxes = []  # Liste für ausgeschnittene Boxen

for box in results.xyxy[0].cpu().numpy():
    cnt=cnt+1
    box = [float(i) for i in box]
    xmin, ymin, xmax, ymax, conf, cls = box
    
    region = image[int(ymin):int(ymax), int(xmin):int(xmax)]
    cvt = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(cvt, 13)
    
    _, otsu_thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(otsu_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    con_array = []
    
    # Sortiere Konturen nach Fläche
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Nutze nur die größte Kontur Aus jedem Bereich
    if contours:
        contour = contours[0]
        
        # Berechne das rotierte Rechteck
        rect = cv2.minAreaRect(contour)
        box_points = cv2.boxPoints(rect)
        box_points = np.int0(box_points)
        
        # Offset basierend auf den Box-Koordinaten
        box_points[:, 0] += int(xmin)
        box_points[:, 1] += int(ymin)
        # Zeichnen der Roten Box um die Contur
        print(f'box_points {box_points}')
        cv2.drawContours(output, [box_points], 0, (0, 0, 255), 2)
        
        x_coords = [point[0] for point in box_points]
        print(f'x_coords {x_coords}')
        y_coords = [point[1] for point in box_points]
        print(f'y_coords {y_coords}')

        # Finde die Begrenzungen des Rechtecks
        x_coords_min, x_coords_max = min(x_coords), max(x_coords)
        y_coords_min, y_coords_max = min(y_coords), max(y_coords)
        height = y_coords_max - y_coords_min
        width = x_coords_max - x_coords_min
        
        # Schneide den Bildausschnitt basierend auf den Begrenzungen aus
        cropped_image = image[y_coords_min:y_coords_max, x_coords_min:x_coords_max]
        
        if height < width:
            cropped_boxes.append(cropped_image)
        else:
            cropped_boxes.append(cv2.rotate(cropped_image, cv2.ROTATE_90_COUNTERCLOCKWISE))
   
    # Draw Green Rectangles 
    output = cv2.rectangle(output, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
    output = cv2.putText(output, labels[int(cls)], (int(xmin), int(ymin) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                         (0, 255, 0), 1, cv2.LINE_AA)    

cv2.imshow('Test34', cropped_boxes[0])
print(f'cnt: {cnt}')
# Draw Objects on Steady Background
black_BG = ip.standardize_images(cropped_boxes)

for i, image in enumerate(black_BG):

    path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\Scanned Data\\Part{}.png'.format(i)
    cv2.imwrite(path, image)
    
cv2.imshow("Predicted Objects", output)
cv2.waitKey(0)
cv2.destroyAllWindows()



C:\Users\doett\AppData\Local\Temp\ipykernel_8396\3928964719.py:33: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box_points = np.int0(box_points)


box_points [[476 510]
 [502 492]
 [631 687]
 [605 704]]
x_coords [476, 502, 631, 605]
y_coords [510, 492, 687, 704]
box_points [[1145  541]
 [1221  399]
 [1251  414]
 [1175  557]]
x_coords [1145, 1221, 1251, 1175]
y_coords [541, 399, 414, 557]
box_points [[ 886  299]
 [1010  116]
 [1057  147]
 [ 932  331]]
x_coords [886, 1010, 1057, 932]
y_coords [299, 116, 147, 331]
box_points [[750 255]
 [922 128]
 [932 141]
 [760 269]]
x_coords [750, 922, 932, 760]
y_coords [255, 128, 141, 269]
box_points [[355  93]
 [374  73]
 [471 169]
 [451 189]]
x_coords [355, 374, 471, 451]
y_coords [93, 73, 169, 189]
box_points [[320 384]
 [483 294]
 [489 305]
 [326 395]]
x_coords [320, 483, 489, 326]
y_coords [384, 294, 305, 395]
box_points [[682 103]
 [704 101]
 [764 486]
 [741 490]]
x_coords [682, 704, 764, 741]
y_coords [103, 101, 486, 490]
box_points [[797 609]
 [871 463]
 [907 481]
 [833 627]]
x_coords [797, 871, 907, 833]
y_coords [609, 463, 481, 627]
box_points [[587 520]
 [597 515]
 [677 688]
 [667 69

In [117]:
image = cv2.imread(image_path)
output = image.copy()
results = model(image)


# Extract Conventional

In [6]:
DataFolderPath = 'Testing/Scanned Data'

In [7]:
image = cv2.imread(image_path)
output = image.copy()
results = model(image)

In [8]:
preprocessed_image = ip.image_preprocessing(image)
cv2.imshow('Test',preprocessed_image)
contours, rectangles, bounding_boxes, object_images = ip.get_objects_in_preprocessed_image(preprocessed_image,
                                                                                             smaller_image_area=True)
black_BG = ip.standardize_images(object_images,224)

#cv2.imshow('test',black_BG[0])

canvas_image = cv2.drawContours(preprocessed_image, bounding_boxes, -1, (0, 0, 255), 2)
cv2.imshow('canvas_image',canvas_image)

cv2.waitKey(0)
cv2.destroyAllWindows() 


    